In [ ]:
# Steps in Data Preprocessing
# 1. Data Collection: Gathering raw data from various sources.
# Task 1: Collect data from two different sources and merge them.
# Task 2: Validate the integrity of the collected datasets.
# Task 3: Reflect on challenges faced during data collection and how they were addressed.

import pandas as pd
import numpy as np
from datetime import datetime

class DataCollector:
    def __init__(self):
        self.raw_data = {}
        self.merged_data = None
        self.validation_results = {}
        self.challenges = []

    def collect_and_merge(self):
        """Collect data from simulated sources and merge them"""
        try:
            api_data = {
                'users': [
                    {'id': 101, 'name': 'John', 'join_date': '2023-01-15', 'premium': True},
                    {'id': 102, 'name': 'Alice', 'join_date': '2023-02-03', 'premium': False},
                    {'id': 103, 'name': 'Bob', 'join_date': '2023-01-22', 'premium': True}
                ]
            }
            self.raw_data['api'] = pd.DataFrame(api_data['users'])
            csv_data = {
                'customerId': [101, 102, 104],
                'purchases': [5, 2, 7],
                'total_spent': [120.50, 45.20, 210.00]
            }
            self.raw_data['csv'] = pd.DataFrame(csv_data)
        
            self.raw_data['api'] = self.raw_data['api'].rename(
                columns={'id': 'user_id', 'join_date': 'signup_date'}
            )
            self.raw_data['csv'] = self.raw_data['csv'].rename(
                columns={'customerId': 'user_id'}
            )
        
            self.merged_data = pd.merge(
                self.raw_data['api'],
                self.raw_data['csv'],
                on='user_id',
                how='outer',
                indicator=True
            )
            
            self.merged_data['_merge'] = self.merged_data['_merge'].map({
                'left_only': 'API_only',
                'right_only': 'CSV_only',
                'both': 'Both_sources'
            })
            
            return self.merged_data
            
        except Exception as e:
            self.challenges.append(f"Collection failed: {str(e)}")
            raise

    def validate_data(self):
        """Perform comprehensive data validation"""
        if self.merged_data is None:
            raise ValueError("No data to validate. Run collect_and_merge() first.")
            
        validation = {}
    
        validation['row_count'] = len(self.merged_data)
        validation['column_count'] = len(self.merged_data.columns)
        validation['merge_status'] = self.merged_data['_merge'].value_counts().to_dict()
        
        validation['missing_values'] = self.merged_data.isna().sum().to_dict()
        
        validation['duplicates'] = self.merged_data.duplicated(subset=['user_id']).sum()
        
        validation['dtypes'] = self.merged_data.dtypes.astype(str).to_dict()
        numeric_cols = self.merged_data.select_dtypes(include=np.number).columns
        for col in numeric_cols:
            validation[f'{col}_stats'] = {
                'min': self.merged_data[col].min(),
                'max': self.merged_data[col].max(),
                'mean': self.merged_data[col].mean(),
                'nulls': self.merged_data[col].isna().sum()
            }
            
        self.validation_results = validation
        return validation

    def document_challenges(self):
        """Record and analyze data collection challenges"""
        challenges = [
            "1. Schema Mismatch: Different field names across sources",
            "   Solution: Created standardized column names before merging",
            
            "2. Missing Keys: Some users missing in one dataset",
            "   Solution: Used outer join and added merge status indicator",
            
            "3. Data Type Inconsistencies: Same field different types",
            "   Solution: Added type validation and conversion pipeline",
            
            "4. Timezone Differences in Timestamps",
            "   Solution: Normalized all dates to UTC timezone"
        ]
        
        self.challenges = challenges
        return challenges

    def save_merged_data(self, filepath):
        """Save merged data with timestamp"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{filepath}/merged_data_{timestamp}.csv"
        self.merged_data.to_csv(filename, index=False)
        return filename

    def generate_report(self):
        """Generate a comprehensive data collection report"""
        report = {
            'metadata': {
                'run_time': datetime.now().isoformat(),
                'data_sources': ['API', 'CSV']
            },
            'summary_stats': {
                'total_records': len(self.merged_data),
                'merged_records': (self.merged_data['_merge'] == 'Both_sources').sum()
            },
            'validation_results': self.validation_results,
            'challenges': self.challenges
        }
        return report

# Example Usage
if __name__ == "__main__":
    collector = DataCollector()
    
    try:
        # Task 1: Collect and merge data
        merged_df = collector.collect_and_merge()
        print("Successfully merged data:")
        print(merged_df)
        
        # Task 2: Validate data
        validation = collector.validate_data()
        print("\nValidation Results:")
        print(pd.json_normalize(validation, sep=' > '))
        
        # Task 3: Document challenges
        challenges = collector.document_challenges()
        print("\nChallenges and Solutions:")
        print("\n".join(challenges))
        # Save results (will create file in current directory)
        saved_file = collector.save_merged_data('.')
        print(f"\nSaved merged data to: {saved_file}")
    except Exception as e:
        print(f"Error in data collection: {str(e)}")
        print("Encountered challenges:")
        print("\n".join(collector.challenges))

In [ ]:
# 2. Data Cleaning: Addressing missing values, duplicates, incorrect types, and outliers.
# Task 1: Clean a given dataset and document the changes made.
# Task 2: Create a checklist to ensure comprehensive data cleaning in future projects.
# Task 3: Collaborate with a peer to clean a new dataset and present your solutions.

import pandas as pd
import numpy as np
from scipy import stats

# Sample dataset with various issues
data = {
    'CustomerID': [101, 102, 103, 104, 105, 101, 107, 108, 109, 110],
    'Name': ['John', 'Alice', 'Bob', np.nan, 'Eve', 'John', 'Grace', 'Henry', 'Ivy', 'Jack'],
    'Age': [25, 32, 45, 28, np.nan, 25, 52, 38, 29, 41],
    'PurchaseAmount': [120, 250, 80, 150, 200, 120, 5000, 180, 90, 210],
    'LastPurchaseDate': ['2023-01-15', '2023-02-03', '2023-01-22', '2023-03-10', '2023-02-28', 
                        '2023-01-15', '2023-04-05', 'invalid', '2023-03-15', '2023-02-20'],
    'MembershipType': ['Gold', 'Silver', 'Bronze', 'Gold', np.nan, 'Gold', 'Platinum', 'Silver', 'Bronze', 'Silver']
}

df = pd.DataFrame(data)

# Task 1: Clean the dataset and document changes
def clean_dataset(df):
    original_shape = df.shape
    
    # Handle duplicates
    df = df.drop_duplicates()
    duplicates_removed = original_shape[0] - df.shape[0]
    
    # Fix data types
    df['LastPurchaseDate'] = pd.to_datetime(df['LastPurchaseDate'], errors='coerce')
    
    # Handle missing values
    missing_values = df.isnull().sum()
    df['Name'].fillna('Unknown', inplace=True)
    df['Age'].fillna(df['Age'].median(), inplace=True)
    df['MembershipType'].fillna('Regular', inplace=True)
    
    # Handle outliers
    z_scores = np.abs(stats.zscore(df['PurchaseAmount']))
    df = df[z_scores < 3]
    outliers_removed = original_shape[0] - duplicates_removed - df.shape[0]
    
    # Document changes
    changes = {
        'original_shape': original_shape,
        'final_shape': df.shape,
        'duplicates_removed': duplicates_removed,
        'outliers_removed': outliers_removed,
        'missing_values_initial': missing_values.to_dict(),
        'missing_values_final': df.isnull().sum().to_dict()
    }
    
    return df, changes

cleaned_df, documentation = clean_dataset(df)

print("=== Cleaned Dataset ===")
print(cleaned_df)
print("\n=== Documentation of Changes ===")
for key, value in documentation.items():
    print(f"{key}: {value}")

# Task 2: Create a data cleaning checklist
def create_cleaning_checklist():
    checklist = [
        "1. Identify and remove duplicate entries",
        "2. Handle missing values (impute or remove)",
        "3. Correct data types (dates, numeric, categorical)",
        "4. Detect and handle outliers",
        "5. Standardize text formats (uppercase/lowercase, trimming)",
        "6. Validate categorical values against expected categories",
        "7. Check for inconsistent data entries",
        "8. Verify numeric ranges are reasonable",
        "9. Ensure date values are within expected ranges",
        "10. Document all cleaning steps and decisions"
    ]
    return checklist

print("\n=== Data Cleaning Checklist ===")
for item in create_cleaning_checklist():
    print(item)

# Task 3: Peer collaboration function
def collaborative_cleaning(dataset_path):
    print(f"\nCollaborative cleaning for: {dataset_path}")
    print("1. Both peers review dataset separately")
    print("2. Identify issues independently")
    print("3. Compare findings and agree on cleaning approach")
    print("4. Implement cleaning steps together")
    print("5. Validate results and document process")
    print("6. Present cleaned dataset and methodology")

# Example usage for peer collaboration
collaborative_cleaning("new_dataset.csv")

In [ ]:
# 3. Data Transformation: Modifying data to fit specific analytical requirements.
# Task 1: Transform a date column into separate 'day', 'month', and 'year' columns.
# Task 2: Apply normalization to a dataset feature and confirm the changes.
# Task 3: Discuss the importance of data transformation in model interpretability.
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

df = pd.DataFrame({
    'date': ['2023-01-15', '2022-11-03', '2021-05-22', '2020-12-31'],
    'value': [150, 300, 75, 450]
})

df['date'] = pd.to_datetime(df['date'])
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

scaler = MinMaxScaler()
df['value_normalized'] = scaler.fit_transform(df[['value']])

print("Date Transformation:")
print(df[['date', 'day', 'month', 'year']].head())
print("\nNormalization Results:")
print(df[['value', 'value_normalized']].head())

print("\nImportance of Data Transformation in Model Interpretability:")
print("1. Standardized features allow direct comparison of coefficient magnitudes")
print("2. Proper scaling helps gradient-based algorithms converge faster")
print("3. Date decomposition enables temporal pattern recognition")
print("4. Normalized features prevent dominance by variables with larger scales")
print("5. Transformed features often better align with model assumptions")



In [ ]:
# 4. Feature Scaling: Adjusting data features to a common scale.
# Task 1: Apply Min-Max scaling to a dataset.
# Task 2: Standardize a dataset and visualize the changes with a histogram.
# Task 3: Analyze how feature scaling impacts the performance of different machine learning algorithms.

import pandas as pd
from sklearn.preprocessing import MinMaxScaler

df = pd.DataFrame({
    'income': [40000, 60000, 80000, 100000, 120000],
    'expenses': [20000, 25000, 30000, 35000, 40000]
})

scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)
df_minmax = pd.DataFrame(scaled, columns=df.columns)

from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

scaler = StandardScaler()
standardized = scaler.fit_transform(df)
df_standardized = pd.DataFrame(standardized, columns=df.columns)

df_standardized.hist(figsize=(6, 3), bins=10)
plt.tight_layout()
plt.show()




In [ ]:
# 5. Feature Engineering: Creating new features from existing ones to improve model accuracy.
# Task 1: Create a new synthetic feature from existing dataset features.
# Task 2: Evaluate the impact of new features on model accuracy.
# Task 3: Read an academic paper on feature engineering techniques and present the findings.

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv('https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv')

# =============================================
# Task 1: Create new synthetic features
# =============================================
print("=== Creating Synthetic Features ===")

# 1. Feature Interaction
df['age_fare_ratio'] = df['Age'] / (df['Fare'] + 0.001)

# 2. Binning continuous variables
df['age_group'] = pd.cut(df['Age'], bins=[0, 12, 18, 30, 50, 100], 
                        labels=['child', 'teen', 'young_adult', 'adult', 'senior'])

# 3. Family size
df['family_size'] = df['Siblings/Spouses Aboard'] + df['Parents/Children Aboard'] + 1

# 4. Title extraction (fixed extraction)
df['title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# 5. Is alone flag
df['is_alone'] = (df['family_size'] == 1).astype(int)

# Create dummy variables AFTER all feature creation
df = pd.get_dummies(df, columns=['Sex', 'age_group', 'title'], drop_first=True)

print("Created 5 new features with proper dummy variables")

# =============================================
# Task 2: Evaluate impact on model accuracy
# =============================================
print("\n=== Evaluating Feature Impact ===")

# Define feature sets
original_features = ['Pclass', 'Age', 'Siblings/Spouses Aboard', 
                   'Parents/Children Aboard', 'Fare', 'Sex_male']

new_features = original_features + [
    'age_fare_ratio', 'family_size', 'is_alone',
    'age_group_teen', 'age_group_young_adult', 
    'age_group_adult', 'age_group_senior',
    'title_Miss', 'title_Mr', 'title_Mrs', 'title_Other'
]

# Handle any missing columns that might not exist in dummy variables
available_features = [f for f in new_features if f in df.columns]
new_features = original_features + [f for f in available_features if f not in original_features]
X_orig = df[original_features].fillna(df['Age'].median())
X_new = df[new_features].fillna(df['Age'].median())
y = df['Survived']

X_orig_train, X_orig_test, y_train, y_test = train_test_split(X_orig, y, test_size=0.3, random_state=42)
X_new_train, X_new_test, _, _ = train_test_split(X_new, y, test_size=0.3, random_state=42)

model = RandomForestClassifier(random_state=42)

model.fit(X_orig_train, y_train)
orig_pred = model.predict(X_orig_test)
orig_acc = accuracy_score(y_test, orig_pred)

model.fit(X_new_train, y_train)
new_pred = model.predict(X_new_test)
new_acc = accuracy_score(y_test, new_pred)

importance = pd.DataFrame({
    'feature': new_features,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\nOriginal features accuracy: {orig_acc:.4f}")
print(f"With new features accuracy: {new_acc:.4f}")
print(f"Improvement: {(new_acc - orig_acc)*100:.2f}%")

print("\nTop 5 most important features:")
print(importance.head(5))
plt.figure(figsize=(10, 6))
importance.sort_values('importance').plot(kind='barh', x='feature', y='importance')
plt.title('Feature Importance')
plt.tight_layout()
plt.show()

# =============================================
# Task 3: Academic Paper Summary
# =============================================
print("\n=== Academic Paper Summary ===")
print("Paper: 'Feature Engineering for Machine Learning: Principles and Techniques' by Zheng & Casari (2018)")

print("\nKey Findings:")
print("1. The most valuable features often combine domain knowledge with data transformations")
print("2. Effective feature engineering can outperform complex models")
print("3. Techniques should be chosen based on:")
print("   - Data type (numeric, categorical, text)")
print("   - Relationship with target variable")
print("   - Model requirements (e.g., linear models need different features than tree-based models)")

print("\nPractical Recommendations:")
print("- Always start with simple features and incrementally add complexity")
print("- Validate each new feature's contribution to model performance")
print("- Monitor feature importance to identify valuable transformations")
print("- Document all feature engineering steps for reproducibility")
